In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

import os
from getpass import getpass
import pandas as pd
import json
import re
from tqdm import tqdm
from sklearn.metrics import classification_report
from json.decoder import JSONDecodeError

import nest_asyncio
from dotenv import load_dotenv
from together import Together

pd.set_option('max_colwidth', 400)

nest_asyncio.apply()

load_dotenv('/app/data_generation/.env', verbose=True)

True

Для простоты мы будем использовать API сервиса Together. При [регистрации](https://www.together.ai/) вы получите бесплатные кредиты на сумму 1$, чего должно быть достаточно для выполнения этой домашней работы. После регистрации вы сможете получить свой персональный API-ключ.

Для добавления переменной API_KEY при использовании load_dotenv, вам нужно:

* Создать файл .env в той же директории, где находится ваш ноутбук, и добавить в него следующую строку:

    `API_KEY=ваш_ключ_api_здесь`
* После этого, когда вы вызываете load_dotenv(), переменная будет загружена в окружение и доступна через os.environ.get('API_KEY').

In [3]:
api_key = os.environ.get('API_KEY')
client = Together(api_key=api_key)

In [4]:
def get_completion(prompt: str, system_prompt=""):
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[
            {
                    "role": "system",
                    "content": system_prompt
            },
            {
                    "role": "user",
                    "content": prompt
            },
    ],
        max_tokens=1000,
        temperature=0.0,
        top_p=0,
        top_k=0,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
        stream=False
    )
    return response.choices[0].message.content

In [5]:
get_completion("Привет!")

'Привет! Как я могу вам помочь сегодня?'

## Предсказание категории

In [6]:
categories = [
 'Ролики и скейтбординг',
 'Настольные игры',
 'Теннис бадминтон пинг-понг',
 'Пейнтбол и страйкбол',
 'Единоборства',
 'Бильярд и боулинг',
 'Фитнес и тренажёры',
 'Туризм',
 'Игры с мячом',
 'Зимние виды спорта',
 'Дайвинг и водный спорт',
 'Спортивное питание'
]

system_prompt = """Твоя задача - выбрать наиболее подходящую категорию (даже если информации недостаточно - 
необходимо строго одну из списка) к товару пользователя по его текстовому описанию. 
Верни свой ответ в формате JSON следующего вида - {"category": "answer"}
Вот список категорий, из которых ты должен выбрать одну - 
""" + ';'.join(categories)

In [7]:
# test_df лежит на каггле в разделе Data
test_df = pd.read_csv('test_df_llm_generation.csv')[['title', 'item_id']]
test_df.sample(10)

,title,item_id
108,Детские ролики и защита,108
123,Обруч халахуп,123
90,Кимоно Kudo wear б/у,90
3,Юнион 27” abec 7 - полный аналог Penny Nikel,3
28,Поповский кий Баринова черный эбен,28
137,Байдарка Свирь-Н,137
101,Темляки Swix triac L на доставке,101
93,Шарики для пинг-понга,93
107,Разгрузочный жилет / разгрузка,107
62,Санки тимка 5 универсал с колесами,62


In [16]:
def process_dataframe_with_cache(test_df, system_prompt, cache_file='answer_cache.json'):
    key_to_answer = {}
    if os.path.exists(cache_file):
        with open(cache_file, 'r', encoding='utf-8') as f:
            key_to_answer = json.load(f)
    answers = []
    
    for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
        title = row['title']
        item_id = row['item_id']
        key = f"{system_prompt}_{title}"
        
        if key in key_to_answer:
            answer = key_to_answer[key]
        else:
            prompt = system_prompt + f"\n<text>{title}</text>"
            answer = get_completion(prompt)
            key_to_answer[key] = answer
            # Сохраняем кэш после каждого нового ответа
            with open(cache_file, 'w', encoding='utf-8') as f:
                json.dump(key_to_answer, f, ensure_ascii=False, indent=2)
        answers.append(answer)
    
    return answers

In [17]:
answers = process_dataframe_with_cache(test_df, system_prompt)
predicted_categories = []
for answer in answers:
    try:
        answer = json.loads(answer)
        predicted_category = answer['category']
    except JSONDecodeError as e:
        predicted_category = 'other'
    predicted_categories.append(predicted_category)

100%|██████████| 147/147 [01:26<00:00,  1.71it/s]


In [35]:
test_df_gpt = test_df.copy()
test_df_gpt['category'] = predicted_categories
test_df_gpt[['item_id', 'category']].to_csv('test_df_to_upload.csv', index=False)

### Ваше решение